In [1]:
import pandas as pd
import numpy as np
import nltk
import string
import re
import csv
import Sastrawi
from string import digits

# Preprocessing

In [2]:
#ambil data dri tgl 2 agustus 2022
data= pd.read_csv('dataulasan.csv',sep=';',encoding='utf-8') #import datanya
df=pd.DataFrame(data[['label','content']]) #mengambil data yang diperlukan saja
df

,label,content
0,Positif,Terimakasih!!!!!!!!!!!!!
1,Positif,Mantap betul lah
2,Positif,goodjob
3,Positif,Sangat mudah diakses
4,Positif,Siip
...,...,...
995,Negatif,"Minta di update tapi ngak bisa, gimana sih"
996,Negatif,Login susah
997,Negatif,Gak bisa daftar
998,Negatif,"aplikasi kont.. bikin emosi,masa tiap login wa..."


In [3]:
#cleanning :membersihkan data dari simbol,angka,emoji dan 1huruf
def cleanning(content):
    remove_punctuation= "".join([c for c in content if c not in string.punctuation])
    remove_number = ''.join([i for i in remove_punctuation if not i.isdigit()])
    emoj = re.compile("["
        u"\U00002700-\U000027BF"  # Dingbats
        u"\U0001F600-\U0001F64F"  # Emoticons
        u"\U00002600-\U000026FF"  # Miscellaneous Symbols
        u"\U0001F300-\U0001F5FF"  # Miscellaneous Symbols And Pictographs
        u"\U0001F900-\U0001F9FF"  # Supplemental Symbols and Pictographs
        u"\U0001FA70-\U0001FAFF"  # Symbols and Pictographs Extended-A
        u"\U0001F680-\U0001F6FF"  # Transport and Map Symbols
                      "]+", re.UNICODE)
    remove_emoji= re.sub(emoj, '', remove_number)
    remove_single_char=re.sub(r"\b[a-zA-Z]\b", "", remove_emoji)
    return remove_single_char
df['hasil_cleanning']=df['content'].apply(lambda x: cleanning(x))
df

,label,content,hasil_cleanning
0,Positif,Terimakasih!!!!!!!!!!!!!,Terimakasih
1,Positif,Mantap betul lah,Mantap betul lah
2,Positif,goodjob,goodjob
3,Positif,Sangat mudah diakses,Sangat mudah diakses
4,Positif,Siip,Siip
...,...,...,...
995,Negatif,"Minta di update tapi ngak bisa, gimana sih",Minta di update tapi ngak bisa gimana sih
996,Negatif,Login susah,Login susah
997,Negatif,Gak bisa daftar,Gak bisa daftar
998,Negatif,"aplikasi kont.. bikin emosi,masa tiap login wa...",aplikasi kont bikin emosimasa tiap login waktu...


In [4]:
#proses casefolding:mengubah huruf kecil menjadi huruf besar
def casefolding(hasil_cleanning):
    hasil_casefolding=hasil_cleanning.lower()
    return hasil_casefolding
df['hasil_casefolding']=df['hasil_cleanning'].apply(lambda x: casefolding(x))
df[['hasil_cleanning','hasil_casefolding']]

,hasil_cleanning,hasil_casefolding
0,Terimakasih,terimakasih
1,Mantap betul lah,mantap betul lah
2,goodjob,goodjob
3,Sangat mudah diakses,sangat mudah diakses
4,Siip,siip
...,...,...
995,Minta di update tapi ngak bisa gimana sih,minta di update tapi ngak bisa gimana sih
996,Login susah,login susah
997,Gak bisa daftar,gak bisa daftar
998,aplikasi kont bikin emosimasa tiap login waktu...,aplikasi kont bikin emosimasa tiap login waktu...


In [5]:
#proses tokenization:memecah kalimat menjadi kata
from nltk.tokenize import word_tokenize
def tokenization(hasil_stemming):
    token=word_tokenize(hasil_stemming)
    return token
df['hasil_token']=df['hasil_casefolding'].apply(lambda x: tokenization(x))
df[['hasil_casefolding','hasil_token']]

,hasil_casefolding,hasil_token
0,terimakasih,[terimakasih]
1,mantap betul lah,"[mantap, betul, lah]"
2,goodjob,[goodjob]
3,sangat mudah diakses,"[sangat, mudah, diakses]"
4,siip,[siip]
...,...,...
995,minta di update tapi ngak bisa gimana sih,"[minta, di, update, tapi, ngak, bisa, gimana, ..."
996,login susah,"[login, susah]"
997,gak bisa daftar,"[gak, bisa, daftar]"
998,aplikasi kont bikin emosimasa tiap login waktu...,"[aplikasi, kont, bikin, emosimasa, tiap, login..."


In [6]:
#stopword removal
from nltk.corpus import stopwords
nltk.download('stopwords')
stop = stopwords.words('indonesian')
txt_stopword = pd.read_csv("stopwords.txt", names= ["stopwords"], header = None)
stop.extend(txt_stopword["stopwords"][0].split(' '))
def stopword (hasil_token):
    return [word for word in hasil_token if word not in stop]
df['hasil_stopword']=df['hasil_token'].apply(lambda x: stopword(x))
df[['hasil_token','hasil_stopword']]

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\MEI\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


,hasil_token,hasil_stopword
0,[terimakasih],[terimakasih]
1,"[mantap, betul, lah]",[mantap]
2,[goodjob],[]
3,"[sangat, mudah, diakses]","[mudah, diakses]"
4,[siip],[]
...,...,...
995,"[minta, di, update, tapi, ngak, bisa, gimana, ...",[]
996,"[login, susah]",[susah]
997,"[gak, bisa, daftar]",[daftar]
998,"[aplikasi, kont, bikin, emosimasa, tiap, login...","[aplikasi, sesi, habis, suruh, ulang, keluarny..."


In [7]:
#stemming:menghilangkan imbuhan kata
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
import swifter
factory=StemmerFactory()
stemmer=factory.create_stemmer()

#stemmed 
def stemmed_wrapper(term):
    return stemmer.stem(term)
term_dict={}
for document in df['hasil_stopword']:
    for term in document:
        if term not in term_dict:
            term_dict[term]=''

for term in term_dict:
    term_dict[term]=stemmed_wrapper(term)
    
#aplikasikan stemming pada dataframe 
def stemming(document):
    return[term_dict[term] for term in document]
df['hasil_stemming']=df['hasil_stopword'].apply(lambda x: stemming(x))
df[['content','hasil_stemming']]

,content,hasil_stemming
0,Terimakasih!!!!!!!!!!!!!,[terimakasih]
1,Mantap betul lah,[mantap]
2,goodjob,[]
3,Sangat mudah diakses,"[mudah, akses]"
4,Siip,[]
...,...,...
995,"Minta di update tapi ngak bisa, gimana sih",[]
996,Login susah,[susah]
997,Gak bisa daftar,[daftar]
998,"aplikasi kont.. bikin emosi,masa tiap login wa...","[aplikasi, sesi, habis, suruh, ulang, keluar, ..."


In [8]:
#menghapus ulasan yang bernilai kosong
def remove_comments(df):
  #ngecek ulasan yang bernilai zero
  zero_length_comments = df[df["hasil_stemming"].map(len) == 0]
  # mengambil ulasan yang bernilai kosong 
  zero_length_comments_index = [ind for ind in zero_length_comments.index]
  # membuang data yang bernilai kosong 
  df.drop(zero_length_comments_index, inplace = True)
  return df
df = remove_comments(df)
df[['content','hasil_stemming']]

,content,hasil_stemming
0,Terimakasih!!!!!!!!!!!!!,[terimakasih]
1,Mantap betul lah,[mantap]
3,Sangat mudah diakses,"[mudah, akses]"
5,Anda mesti bertanggung jawab atas semua keboco...,"[mesti, tanggung, bocor, data, pribadi, jabat,..."
8,Sangat aman,[aman]
...,...,...
991,"ini kenapa ya,udh di daftar tp gk bisa log in,...","[daftar, muncul, sesi, hapus, unduh]"
992,Mantap,[mantap]
996,Login susah,[susah]
997,Gak bisa daftar,[daftar]


In [9]:
#menentukan tipe data dri setiap kolom
df=df.astype({'hasil_stemming':'string'})
df=df.astype({'label':'category'})


# MENGHITUNG TERM FREQUENCY DATA 

In [10]:
#membuat variabel baru untuk menyimpan data frame bagian data latih
df=pd.DataFrame(df[['hasil_stemming','label']])
#menghitung banyaknya data latih 
n_data=len(df)
print('Banyak Data:',n_data)
#menghitung banyak data yang berlabel negatif
neg_label=df[(df['label']=='Negatif')]
print('Banyak Data Latih yang Negatif adalah',len(neg_label))
#menghitung banyak data yang berlabel positif 
pos_label=df[(df['label']=='Positif')]
print('Banyak Data Latih yang Positif adalah',len(pos_label))

Banyak Data: 739
Banyak Data Latih yang Negatif adalah 506
Banyak Data Latih yang Positif adalah 233


# TF POSITIF

In [11]:
#mengambil term data yang positif 
from sklearn.feature_extraction.text import CountVectorizer
positif=[df['hasil_stemming']for index,df in df.iterrows() if df['label']=='Positif']
#menghitung frekuensi term positif 
vec_pos=CountVectorizer()
x_pos=vec_pos.fit_transform(positif)
tabel_pos=pd.DataFrame(x_pos.toarray(), columns=vec_pos.get_feature_names_out())
tabel_pos

,air,akses,akun,akurat,alam,aman,ambil,antar,antri,aplikasi,...,tulis,ubah,ulang,vaksin,vaksinasi,versi,virus,wajib,warga,zaman
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
228,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
229,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
230,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
231,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [12]:
#menghtiung total kemunculan terms data positif 
word_list_pos=vec_pos.get_feature_names_out();
count_list_pos=x_pos.toarray().sum(axis=0)
freq_pos=dict(zip(word_list_pos,count_list_pos))
word_list_pos
freq_pos

{'air': 1,
 'akses': 2,
 'akun': 2,
 'akurat': 1,
 'alam': 1,
 'aman': 3,
 'ambil': 1,
 'antar': 1,
 'antri': 3,
 'aplikasi': 21,
 'at': 1,
 'bagus': 48,
 'bahar': 1,
 'baharu': 3,
 'baik': 5,
 'bantu': 43,
 'baru': 6,
 'batu': 1,
 'bawa': 1,
 'bebas': 1,
 'beraktifitas': 1,
 'berkah': 1,
 'bilang': 1,
 'bintang': 1,
 'booster': 1,
 'bpjs': 1,
 'buka': 2,
 'butuh': 1,
 'cari': 1,
 'cepat': 9,
 'cetak': 1,
 'coba': 4,
 'copot': 1,
 'covid': 5,
 'daftar': 4,
 'data': 4,
 'dingin': 2,
 'disiplin': 1,
 'efektif': 1,
 'email': 2,
 'enak': 1,
 'erti': 1,
 'fitur': 2,
 'gagal': 1,
 'gampang': 1,
 'gigil': 1,
 'guna': 7,
 'habis': 1,
 'harap': 2,
 'hasil': 1,
 'hidup': 1,
 'hilang': 1,
 'hindar': 1,
 'indonesia': 4,
 'informasi': 2,
 'iur': 2,
 'jaga': 2,
 'jalan': 4,
 'jam': 2,
 'jarak': 1,
 'kartu': 1,
 'kasih': 3,
 'keluh': 1,
 'keren': 3,
 'kerja': 5,
 'kertas': 1,
 'klaim': 1,
 'kondisi': 1,
 'kontrol': 1,
 'kosong': 1,
 'kuat': 1,
 'kunjung': 1,
 'kurang': 1,
 'lahir': 1,
 'lancar': 5,
 

# TF NEGATIF

In [13]:
#mengambil term data yang negatif 
from sklearn.feature_extraction.text import CountVectorizer
negatif=[df['hasil_stemming'] for index,df in df.iterrows() if df['label']=='Negatif']
#menghitung frekuensi term negatif 
vec_neg=CountVectorizer()
x_neg=vec_neg.fit_transform(negatif)
tabel_neg=pd.DataFrame(x_neg.toarray(), columns=vec_neg.get_feature_names_out())
tabel_neg

,akses,aktivitas,akun,akurat,alam,alamat,alami,alih,aman,ambil,...,vaksinasi,valid,verifikasi,versi,via,wajib,warga,web,webnya,website
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
502,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
503,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
504,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
#menghitung total kemunculan data negatif
word_list_neg=vec_neg.get_feature_names_out();
count_list_neg=x_neg.toarray().sum(axis=0)
freq_neg=dict(zip(word_list_neg,count_list_neg))
freq_neg

{'akses': 8,
 'aktivitas': 2,
 'akun': 23,
 'akurat': 1,
 'alam': 1,
 'alamat': 1,
 'alami': 2,
 'alih': 3,
 'aman': 3,
 'ambil': 1,
 'ampun': 5,
 'anak': 2,
 'aneh': 6,
 'anggap': 1,
 'anggar': 1,
 'antigen': 1,
 'aplikasi': 225,
 'arah': 2,
 'atur': 2,
 'bagus': 10,
 'baharu': 27,
 'baik': 39,
 'balik': 1,
 'bansos': 2,
 'bantu': 4,
 'baru': 38,
 'batal': 1,
 'batas': 1,
 'bayang': 1,
 'beda': 1,
 'beli': 1,
 'benah': 1,
 'benci': 1,
 'bentar': 1,
 'berangkat': 5,
 'berat': 2,
 'bersih': 1,
 'biar': 1,
 'bidang': 1,
 'bijak': 1,
 'bikin': 1,
 'bilang': 2,
 'bingung': 3,
 'bintang': 2,
 'blok': 1,
 'bobol': 1,
 'bocor': 3,
 'bodoh': 2,
 'booster': 10,
 'bosan': 2,
 'boster': 4,
 'browser': 1,
 'buang': 1,
 'buka': 72,
 'bukti': 2,
 'bulak': 1,
 'bulan': 1,
 'buruk': 11,
 'butuh': 9,
 'cacat': 3,
 'cair': 2,
 'cari': 4,
 'cepat': 1,
 'cetak': 4,
 'chrome': 3,
 'coba': 29,
 'corona': 1,
 'covid': 2,
 'daftar': 30,
 'dana': 2,
 'dasar': 1,
 'data': 32,
 'dimasukin': 1,
 'dosis': 3,
 'dui

In [15]:
#menghitung total keseluruhan term pada data
docs=[df['hasil_stemming']for index,df in df.iterrows()]
#print(docs)
vec=CountVectorizer()
terms=vec.fit_transform(docs)
total_terms=len(vec.get_feature_names_out())
#mencetak total keseluruhan term pada masing masing label 
print('Banyak Term Positif adalah:',len(word_list_pos))
print('Banyak Term Negatif adalah:',len(word_list_neg))
print('Banyak Terms Pada Data adalah ',total_terms)

Banyak Term Positif adalah: 168
Banyak Term Negatif adalah: 387
Banyak Terms Pada Data adalah  454


# Membentuk model Naive Bayes dengan Data Train

In [16]:
#menghitung prior probability 
prior_neg=len(neg_label)/n_data
print ('Prior Negatif adalah ',prior_neg)
prior_pos=len(pos_label)/n_data
print('Prior Positif adalah ',prior_pos)

Prior Negatif adalah  0.6847090663058186
Prior Positif adalah  0.3152909336941813


In [17]:
#membentuk nilai probabilitas setiap terms negatif
prob_neg=[]
for word,count in zip(word_list_neg,count_list_neg):
    prob_neg.append((count+1)/(len(word_list_neg)+total_terms))
kamus_neg=dict(zip(word_list_neg,prob_neg))
kamus_neg

{'akses': 0.01070154577883472,
 'aktivitas': 0.0035671819262782403,
 'akun': 0.028537455410225922,
 'akurat': 0.0023781212841854932,
 'alam': 0.0023781212841854932,
 'alamat': 0.0023781212841854932,
 'alami': 0.0035671819262782403,
 'alih': 0.0047562425683709865,
 'aman': 0.0047562425683709865,
 'ambil': 0.0023781212841854932,
 'ampun': 0.007134363852556481,
 'anak': 0.0035671819262782403,
 'aneh': 0.008323424494649227,
 'anggap': 0.0023781212841854932,
 'anggar': 0.0023781212841854932,
 'antigen': 0.0023781212841854932,
 'aplikasi': 0.26872770511296074,
 'arah': 0.0035671819262782403,
 'atur': 0.0035671819262782403,
 'bagus': 0.013079667063020214,
 'baharu': 0.03329369797859691,
 'baik': 0.04756242568370987,
 'balik': 0.0023781212841854932,
 'bansos': 0.0035671819262782403,
 'bantu': 0.005945303210463734,
 'baru': 0.04637336504161712,
 'batal': 0.0023781212841854932,
 'batas': 0.0023781212841854932,
 'bayang': 0.0023781212841854932,
 'beda': 0.0023781212841854932,
 'beli': 0.002378121

In [18]:
#membentuk nilai probabilitas setiap terms positif
prob_pos=[]
for word,count in zip(word_list_pos,count_list_pos):
    prob_pos.append((count+1)/(len(word_list_pos)+total_terms))
kamus_pos=dict(zip(word_list_pos,prob_pos))
kamus_pos

{'air': 0.003215434083601286,
 'akses': 0.00482315112540193,
 'akun': 0.00482315112540193,
 'akurat': 0.003215434083601286,
 'alam': 0.003215434083601286,
 'aman': 0.006430868167202572,
 'ambil': 0.003215434083601286,
 'antar': 0.003215434083601286,
 'antri': 0.006430868167202572,
 'aplikasi': 0.03536977491961415,
 'at': 0.003215434083601286,
 'bagus': 0.07877813504823152,
 'bahar': 0.003215434083601286,
 'baharu': 0.006430868167202572,
 'baik': 0.00964630225080386,
 'bantu': 0.0707395498392283,
 'baru': 0.011254019292604502,
 'batu': 0.003215434083601286,
 'bawa': 0.003215434083601286,
 'bebas': 0.003215434083601286,
 'beraktifitas': 0.003215434083601286,
 'berkah': 0.003215434083601286,
 'bilang': 0.003215434083601286,
 'bintang': 0.003215434083601286,
 'booster': 0.003215434083601286,
 'bpjs': 0.003215434083601286,
 'buka': 0.00482315112540193,
 'butuh': 0.003215434083601286,
 'cari': 0.003215434083601286,
 'cepat': 0.01607717041800643,
 'cetak': 0.003215434083601286,
 'coba': 0.008

# INPUT DATA UJI

In [19]:
from nltk.tokenize import word_tokenize
input_text = input('Masukkan sentimen: ')
input_label = input('Masukkan Label Prediksi Manual: ')

Masukkan sentimen: aplikasi bagus
Masukkan Label Prediksi Manual: positif


# PREPROCESSING DATA UJI

In [20]:
new_word_list=cleanning(input_text)
new_word_list=casefolding(new_word_list)
new_word_list=tokenization(new_word_list)
new_word_list=stopword(new_word_list)
new_word_list

['aplikasi', 'bagus']

In [21]:
new_word_list1 = []
fstemming=StemmerFactory()
stemmer=fstemming.create_stemmer()
for token in new_word_list:
    stemmed_word = stemmer.stem(token)
    new_word_list1.append(stemmed_word)
print(new_word_list1)

['aplikasi', 'bagus']


# uji Positif

In [22]:
prob_uji_pos=[]
for word in new_word_list1:
    if word in freq_pos.keys():
        count=freq_pos[word]
        prob_uji_pos.append((count+1)/(len(word_list_pos)+total_terms))
    else:
        count=1
        prob_uji_pos.append(count)
dict(zip(new_word_list1,prob_uji_pos))

{'aplikasi': 0.03536977491961415, 'bagus': 0.07877813504823152}

In [23]:
i=0
prob_positif=1
for i in range(len(prob_uji_pos)):
    prob_positif= prob_positif * prob_uji_pos[i]
print('Nilai Probabilitas Positif: ',prob_positif)
nilai_akhir_positif=prob_positif*prior_pos
print("Nilai akhir positif: ",nilai_akhir_positif)

Nilai Probabilitas Positif:  0.0027863649052429153
Nilai akhir positif:  0.0008785155925867378


# uji negatif

In [24]:
prob_uji_neg=[]
for word in new_word_list1:
    if word in freq_neg.keys():
        count=freq_neg[word]
        prob_uji_neg.append((count+1)/(len(word_list_neg)+total_terms))
    else:
        count=1
        prob_uji_neg.append(count)
dict(zip(new_word_list1,prob_uji_neg))


{'aplikasi': 0.26872770511296074, 'bagus': 0.013079667063020214}

In [25]:
i=0
prob_negatif=1
for i in range(len(prob_uji_neg)):
    prob_negatif= prob_negatif * prob_uji_neg[i]
print('Nilai Probabilitas Negatif: ',prob_negatif)
nilai_akhir_negatif=prob_negatif*prior_neg
print("Nilai akhir negatif: ",nilai_akhir_negatif)

Nilai Probabilitas Negatif:  0.0035148689134870013
Nilai akhir negatif:  0.0024066626119410317


# hasil akhir

In [26]:
#menentukan hasil akhir data uji
print("KALIMAT ASLI:  ",input_text)
print("PREPROCESSING:  ",new_word_list1)
print("LABEL PREDIKSI MANUAL:",input_label)
if nilai_akhir_positif >= nilai_akhir_negatif:
    print("Labelnya dengan Naive Bayes adalah POSITIF")
else :
    print("Labelnya dengan Naive Bayes adalah NEGATIF")

KALIMAT ASLI:   aplikasi bagus
PREPROCESSING:   ['aplikasi', 'bagus']
LABEL PREDIKSI MANUAL: positif
Labelnya dengan Naive Bayes adalah NEGATIF
